In [6]:
%config IPCompleter.greedy = True
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
import matplotlib.pyplot as plt
import pandas as pd
from Two_Class_MNIST import TwoClassMnist as tcm
# Use scikit-learn to grid search the batch size and epochs
import numpy
from sklearn.model_selection import RandomizedSearchCV
from keras.wrappers.scikit_learn import KerasClassifier

In [3]:
x_train, x_test, y_train, y_test, input_shape = tcm.returnDataSet(1, 2, 28)
dataset = ((x_train, y_train), (x_test, y_test))
num_classes = 2
X = x_train
Y = y_train

x_train shape: (12700, 28, 28, 1)
12700 train samples
2167 test samples


In [42]:
# Function to create model, required for KerasClassifier
def create_model(optimizer='adam', learn_rate=0.01,
                 momentum=0, init_mode='uniform', activation='relu',
                dropout_rate=0.0, weight_constraint=0, neuron=1):
    # create model
    model = Sequential()
    model.add(Conv2D(32, kernel_size=(3, 3),
                     activation=activation,
                     input_shape=input_shape))
    model.add(Conv2D(64, (3, 3), activation=activation))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(dropout_rate))
    model.add(Flatten())
    model.add(Dense(128, activation=activation, kernel_initializer=init_mode))
    model.add(Dropout(dropout_rate))
    model.add(Dense(num_classes, activation=activation, kernel_initializer=init_mode))
    
    # Compile model
    optimizer = SGD(lr=learn_rate, momentum=momentum)
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model

In [8]:
# create model
model = KerasClassifier(build_fn=create_model, verbose=1)

# Embed in to dictionary
param_dist = {"batch_size": [8, 16, 32, 64, 128, 256],
              "epochs": [2, 4, 6, 8, 10],
              "optimizer": ['SGD', 'RMSprop', 'Adagrad', 'Adadelta', 'Adam', 'Adamax', 'Nadam'],
              "learn_rate": [0.001, 0.01, 0.1, 0.2, 0.3],
              "momentum": [0.0, 0.2, 0.4, 0.6, 0.8, 0.9],
              "init_mode": ['uniform', 'lecun_uniform', 'normal', 'zero', 'glorot_normal', 'glorot_uniform', 'he_normal', 'he_uniform'],
              "activation": ['softmax', 'softplus', 'softsign', 'relu', 'tanh', 'sigmoid', 'hard_sigmoid', 'linear'],
              "weight_constraint": [1, 2, 3, 4, 5],
              "dropout_rate": [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9],
              "neurons": [1, 5, 10, 15, 20, 25, 30]}

In [18]:
start = time()
randomSearch = RandomizedSearchCV(estimator=model, param_distributions=param_dist, n_jobs=-1)
randomSearchResult = random_search.fit(X, Y)
print((time() - start))

In [23]:
from time import time
start = time()

end = (time() - start)
print("time: ", (time() - start))

time:  0.00016307830810546875


In [21]:
# summarize results

print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

------------ batch size: 8 and epoch size: 1 -------------
Train on 12700 samples, validate on 2167 samples
Epoch 1/1
12700/12700 [==============================] - 178s 14ms/step - loss: 0.0020 - acc: 0.9997 - val_loss: 0.0059 - val_acc: 0.9986
------------ batch size: 8 and epoch size: 1 -------------
Train on 12700 samples, validate on 2167 samples
Epoch 1/1
12700/12700 [==============================] - 175s 14ms/step - loss: 0.0017 - acc: 0.9997 - val_loss: 0.0106 - val_acc: 0.9982
------------ batch size: 32 and epoch size: 1 -------------
Train on 12700 samples, validate on 2167 samples
Epoch 1/1
12700/12700 [==============================] - 120s 9ms/step - loss: 0.0019 - acc: 0.9997 - val_loss: 0.0075 - val_acc: 0.9982
------------ batch size: 32 and epoch size: 1 -------------
Train on 12700 samples, validate on 2167 samples
Epoch 1/1
12700/12700 [==============================] - 121s 10ms/step - loss: 0.0016 - acc: 0.9998 - val_loss: 0.0036 - val_acc: 0.9986
